In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup, NavigableString
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import json
import os
from dotenv import load_dotenv
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import TimeoutException
from seleniumwire.utils import decode as sw_decode

from webdriver_manager.chrome import ChromeDriverManager
from fake_headers import Headers
import time
import numpy as np
import re

load_dotenv()
API_KEY = os.getenv("API_KEY")

In [2]:
def chrome_driver(webdriver_path=None):
    # Generate a fake user agent
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

    ua = user_agent_rotator.get_random_user_agent()

    browser_options = ChromeOptions()
    # browser_options.add_argument("--headless")
    # browser_options.add_argument("--incognito")
    # browser_options.add_argument("--log-level=3")
    # browser_options.add_argument("--disable-gpu")
    # browser_options.add_argument("--disable-extensions")
    # browser_options.add_argument("--disable-notifications")
    # browser_options.add_argument("--disable-popup-blocking")
    browser_options.add_argument(f"user-agent={ua}")

    if webdriver_path is not None:
        driver = webdriver.Chrome(
            service=ChromeService(executable_path=f"{webdriver_path}"),
            options=browser_options
        )
    else:
        driver = webdriver.Chrome(
            service=ChromeService(ChromeDriverManager().install()),
            options=browser_options)

    return driver

def is_social_media_link(url):
    # Regular expression patterns for matching social media URLs
    patterns = {
        'Facebook': r'https?://(www\.)?facebook\.com/[\w.]+/?',
        'X (Twitter)': r'https?://(www\.)?twitter\.com/[\w.]+/?',
        'Instagram': r'https?://(www\.)?instagram\.com/[\w.]+/?',
        'Youtube': r'https?://(www\.)?(youtube\.com|youtu\.be)/[\w.]+/?',
        'TikTok':r'https?://(www\.)?tiktok\.com/@[\w.]+/?'
    }
    # Check each social media pattern to see if it matches the URL
    for platform, pattern in patterns.items():
        if re.match(pattern, url):
            return True, platform
    
    # If no pattern matches, the URL is not recognized as a social media link
    return False, None

In [5]:

# Fetch the webpage
url = "https://www.bundesliga.com/en/bundesliga/clubs"
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all divs with the class 'p-forge-list-item'
container = soup.find('div', attrs={"class":"clubs grid"})
list_items = container.find_all('a')
names = container.find_all('div',attrs={"class": "club"})
# Initialize a list to hold the concatenated URLs
teams = []
concatenated_urls = []
url_to_follow_items = {}
for item, name in zip(list_items,names):
    # Find the first 'a' tag within a 3-stacked div hierarchy
    # Extract the 'href' attribute
    href = item.get('href')
    team = name.text

    
    # Concatenate with the base URL
    concatenated_urls.append(href)
    teams.append(team)

# Print the concatenated URLs
for parent_url, team in zip(concatenated_urls, teams):
    # Fetch the webpage
    try:
        driver = chrome_driver()
        driver.get('https://www.bundesliga.com' + parent_url)
        
        div_element = driver.find_element(By.XPATH, '//*[@id="default"]/div/clubs-page/div/club-detail/article/header/div[2]/div/div/div[1]')

    # Find all <a> tags within that div
        a_tags = div_element.find_elements(By.TAG_NAME, 'a')

        # Initialize a list to hold the hrefs for this parent URL
        href_links = []

        # Extract the 'href' attribute from each 'li' item found
        for item in a_tags:
            href = item.get_attribute('href')
            if href:
                is_valid, platform = is_social_media_link(href)
                if is_valid:
                    href_links.append(href)

        #     script_tag = soup.find('script', id='__NEXT_DATA__')

        #     if script_tag:
        #         # Get the text content of the script tag
        #         script_content = script_tag.string
                
        #         # Load the JSON content into a dictionary
        #         next_data = json.loads(script_content)
                
        #         # Navigate through the dictionary to get to the 'accounts' dictionary
        #         accounts = next_data['props']['pageProps']['initialState']['global']['config']['social']['accounts']
        #         items = []
        #         for _, i in accounts.items():
        #             items.append(i['url'])
        #         href_links = items
        # Associate the list of hrefs with the parent URL in the dictionary
        url_to_follow_items[team] = href_links
        driver.close()
    except Exception:
        print("check", parent_url)
# Print the dictionary
for team, hrefs in url_to_follow_items.items():
    print(f"{team}: {hrefs}")


FC Bayern München: ['https://twitter.com/FCBayernEN', 'https://www.facebook.com/fcbayern.en/', 'https://www.instagram.com/fcbayern/']
Borussia Dortmund: ['https://twitter.com/BlackYellow', 'https://www.facebook.com/BVB', 'https://www.instagram.com/bvb09/']
RB Leipzig: ['https://twitter.com/RBLeipzig_EN', 'https://www.facebook.com/rbleipzig', 'https://www.instagram.com/rbleipzig/?hl=en']
1. FC Union Berlin: ['https://twitter.com/fcunion_en', 'https://www.facebook.com/1.FCUnionBerlin', 'https://www.instagram.com/1.fcunion/']
Sport-Club Freiburg: ['https://twitter.com/scfreiburg', 'https://www.facebook.com/SCFreiburg', 'https://www.instagram.com/scfreiburg/']
Bayer 04 Leverkusen: ['https://twitter.com/bayer04_en', 'https://www.facebook.com/bayer04fussball', 'https://www.instagram.com/bayer04fussball/']
Eintracht Frankfurt: ['https://twitter.com/eintracht_eng', 'https://www.facebook.com/eintracht.eng/', 'https://www.instagram.com/eintrachtfrankfurt/']
VfL Wolfsburg: ['https://twitter.com/V

In [7]:
sorted_teams = dict(sorted(url_to_follow_items.items()))
sorted_teams

{'1. FC Heidenheim 1846': ['https://twitter.com/FCH1846',
  'https://www.facebook.com/FCH1846',
  'https://www.instagram.com/fch_1846/'],
 '1. FC Köln': ['https://twitter.com/fckoeln_en',
  'https://www.facebook.com/fckoeln.en/',
  'https://www.instagram.com/fckoeln/'],
 '1. FC Union Berlin': ['https://twitter.com/fcunion_en',
  'https://www.facebook.com/1.FCUnionBerlin',
  'https://www.instagram.com/1.fcunion/'],
 '1. FSV Mainz 05': ['https://twitter.com/mainz05en',
  'https://www.facebook.com/1fsvmainz05',
  'https://www.instagram.com/1fsvmainz05/'],
 'Bayer 04 Leverkusen': ['https://twitter.com/bayer04_en',
  'https://www.facebook.com/bayer04fussball',
  'https://www.instagram.com/bayer04fussball/'],
 'Borussia Dortmund': ['https://twitter.com/BlackYellow',
  'https://www.facebook.com/BVB',
  'https://www.instagram.com/bvb09/'],
 'Borussia Mönchengladbach': ['https://twitter.com/borussia_en',
  'https://www.facebook.com/borussia.global',
  'https://www.instagram.com/borussia/'],
 'E

In [17]:
ref = pd.read_excel("bundesliga.xlsx")

In [15]:
for (index, row),(key, val)  in zip(ref.iterrows(), sorted_teams.items()):
    try:
        print(key)
        columns = ['Instagram', 'Facebook', 'TikTok','X (Twitter)','Youtube']
        for j in columns:
            for i in val:
                is_valid, platform = is_social_media_link(i)
                if is_valid:
                    ref.at[index, platform] = i
        ref.at[index, 'Unnamed: 0'] = key
    except:
        continue
    # row['Youtube'] = ([i for i in value if "youtube" in str(i)] + [""])[0]
    # row['Instagram'] = ([i for i in value if "instagram" in str(i)] + [""])[0]
    # row['Facebook'] = ([i for i in value if "facebook" in str(i)] + [""])[0]
    # row['TikTok'] = ([i for i in value if "tiktok" in str(i)] + [""])[0]
    # row['X (Twitter)'] = ([i for i in value if "twitter" in str(i)] + [""])[0]


1. FC Heidenheim 1846
1. FC Köln
1. FC Union Berlin
1. FSV Mainz 05
Bayer 04 Leverkusen
Borussia Dortmund
Borussia Mönchengladbach
Eintracht Frankfurt
FC Augsburg
FC Bayern München
RB Leipzig
SV Darmstadt 98
SV Werder Bremen
Sport-Club Freiburg
TSG Hoffenheim
VfB Stuttgart
VfL Bochum 1848
VfL Wolfsburg


In [16]:
ref.to_excel('bundesliga.xlsx', index=False)

In [18]:
#Facebook
def text_to_int(text):
    # Initialize a multiplier
    multiplier = 1
    
    # Check if 'M' is in the text, indicating millions
    if 'na' in text:
        if 'M' in text:
            multiplier = 1000000
            text = text.replace('M (na) follower', '')  # Remove 'M' for conversion
        elif 'K' in text:
            multiplier = 1000
            text = text.replace('K (na) follower', '')
    else:
        if 'M' in text:
            multiplier = 1000000
            text = text.replace('M followers', '')  # Remove 'M' for conversion
        elif 'K' in text:
            multiplier = 1000
            text = text.replace('K followers', '')
    return int(float(text) * multiplier)
def chrome_driver1(webdriver_path=None):
    # Generate a fake user agent
    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

    ua = user_agent_rotator.get_random_user_agent()

    browser_options = ChromeOptions()
    browser_options.add_argument("--headless")
    # browser_options.add_argument("--incognito")
    # browser_options.add_argument("--log-level=3")
    # browser_options.add_argument("--disable-gpu")
    # browser_options.add_argument("--disable-extensions")
    # browser_options.add_argument("--disable-notifications")
    # browser_options.add_argument("--disable-popup-blocking")
    # browser_options.add_argument(f"user-agent={ua}")

    if webdriver_path is not None:
        driver = webdriver.Chrome(
            service=ChromeService(executable_path=f"{webdriver_path}"),
            options=browser_options
        )
    else:
        driver = webdriver.Chrome(
            service=ChromeService(ChromeDriverManager().install()),
            options=browser_options)

    return driver
for index, row in ref.iterrows():
    driver = chrome_driver1()
    driver.get(row['Facebook'])
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    script_tags = soup.find_all('script', attrs={'data-content-len': True})

    # Iterate over these script tags
    script_tags = soup.find_all('script')

    # Iterate over these script tags
    for script in script_tags:
        data_content_len = script.get('data-content-len')
        if data_content_len and int(data_content_len) > 20000:
            try:
                # Parse the script tag's content as JSON
                script_json = json.loads(script.text)
                for i in script_json['require'][0][3][0]['__bbox']['require'][7][3][1]['__bbox']['result']['data']['user']['profile_header_renderer']['user']['profile_social_context']['content']:
                    if i['uri'].endswith("/followers/"):
                        nested_content = i['text']['text']
                        ref.at[index, 'Facebook (followers)'] = text_to_int(nested_content)
                print(row['Facebook'],nested_content)
                break  # Found the target script tag, no need to continue
            except (KeyError, IndexError, json.JSONDecodeError):
                continue
    driver.quit()
# wait = WebDriverWait(driver, 10)  # Wait for up to 10 seconds
# element_present = wait.until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, f'a[href="https://www.facebook.com/Padres/followers/"]'))
# )

# # Now execute the script
# script = """
# var element = document.querySelector('a[href="https://www.facebook.com/Padres/followers/"]');
# return element ? element.textContent : 'Element not found';
# """
# followers_text = driver.execute_script(script)
# followers_text = text_to_int(followers_text)
# print(followers_text)

https://www.facebook.com/FCH1846 49K (na) follower
https://www.facebook.com/fckoeln.en/ 819K (na) follower
https://www.facebook.com/1fsvmainz05 431K (na) follower
https://www.facebook.com/bayer04fussball 3.2M (na) follower
https://www.facebook.com/BVB 15M (na) follower
https://www.facebook.com/borussia.global 1.3M (na) follower
https://www.facebook.com/fcaugsburgenglish/ 287K (na) follower
https://www.facebook.com/fcbayern.en/ 60M (na) follower
https://www.facebook.com/rbleipzig 1.1M (na) follower
https://www.facebook.com/SVDarmstadt1898eV 146K (na) follower
https://www.facebook.com/SCFreiburg 389K (na) follower
https://www.facebook.com/TSGHoffenheim.en/ 352K (na) follower
https://www.facebook.com/VfB 563K (na) follower
https://www.facebook.com/vflwolfsburg.int/ 2.1M (na) follower


In [19]:
ref

,Unnamed: 0,link,YouTube (subscribers),Instagram (followers),Facebook (followers),TikTok (followers),X (Twitter) (followers),Unnamed: 6,Instagram,Facebook,TikTok,X (Twitter),Youtube
0,1. FC Heidenheim 1846,NaN,7840,72633,NaN,56700,40183,NaN,https://www.instagram.com/fch_1846/,https://www.facebook.com/FCH1846,https://www.tiktok.com/@fch_achtzehn_46,https://twitter.com/FCH1846,https://www.youtube.com/channel/UCa3-PdvjgmAb5...
1,1. FC Köln,NaN,104000,573641,NaN,621100,56759,NaN,https://www.instagram.com/fckoeln/,https://www.facebook.com/fckoeln.en/,https://www.tiktok.com/@fckoeln,https://twitter.com/fckoeln_en,https://www.youtube.com/channel/UCQZhvmm636X6I...
2,1. FC Union Berlin,NaN,46000,356274,NaN,117400,38882,NaN,https://www.instagram.com/1.fcunion/,https://www.facebook.com/1.FCUnionBerlin,https://tiktok.com/@fcunion,https://twitter.com/fcunion_en,https://www.youtube.com/channel/UCexHeD0RsxZFZ...
3,1. FSV Mainz 05,NaN,21300,227454,NaN,117400,38039,NaN,https://www.instagram.com/1fsvmainz05/,https://www.facebook.com/1fsvmainz05,https://www.tiktok.com/@fcunion,https://twitter.com/mainz05en,https://www.youtube.com/channel/UCMT1hwsDgOEMh...
4,Bayer 04 Leverkusen,NaN,315000,3413537,NaN,2800000,499721,NaN,https://www.instagram.com/bayer04fussball/,https://www.facebook.com/bayer04fussball,https://www.tiktok.com/@bayer04,https://twitter.com/bayer04_en,https://www.youtube.com/channel/UCSMZmPVql528C...
5,Borussia Dortmund,NaN,1200000,19819284,NaN,8000000,2483069,NaN,https://www.instagram.com/bvb09/,https://www.facebook.com/BVB,https://www.tiktok.com/@bvb,https://twitter.com/BlackYellow,https://www.youtube.com/channel/UCK8rTVgp3-Meb...
6,Borussia Mönchengladbach,NaN,205000,850636,NaN,1200000,182394,NaN,https://www.instagram.com/borussia/,https://www.facebook.com/borussia.global,https://www.tiktok.com/@borussia,https://twitter.com/borussia_en,https://www.youtube.com/channel/UCcAV1UxgHz4cz...
7,Eintracht Frankfurt,NaN,146000,1143593,NaN,1200000,105026,NaN,https://www.instagram.com/eintrachtfrankfurt/,https://www.facebook.com/eintracht.eng/,https://www.tiktok.com/@eintracht,https://twitter.com/eintracht_eng,https://www.youtube.com/channel/UCDFp4bscTFm8h...
8,FC Augsburg,NaN,22200,211790,NaN,220100,42390,NaN,https://www.instagram.com/fcaugsburg1907/,https://www.facebook.com/fcaugsburgenglish/,https://www.tiktok.com/@fcaugsburg,https://twitter.com/FCA_World,https://www.youtube.com/channel/UCMH-8bjKSpk1e...
9,FC Bayern München,NaN,4000000,41636124,NaN,18800000,3415557,NaN,https://www.instagram.com/fcbayern/,https://www.facebook.com/fcbayern.en/,https://www.tiktok.com/@fcbayern,https://twitter.com/FCBayernEN,https://www.youtube.com/channel/UCZkcxFIsqW5ht...
